In [431]:
#CHRISTIAN!
#Obtener Token y Cookies
import requests
from requests.auth import HTTPBasicAuth

val_url= 'https://my300972-api.s4hana.ondemand.com/sap/opu/odata/sap/API_MKT_EXTERNAL_RECMDN_SRV/$metadata'

val_user='INTEGRATION_ECATALOG'
val_pass='HiVNURziqUMkHcbUGZnsDfQDhuoXMiXgoXmY#Y4n'

val_headers = {'x-csrf-token': 'Fetch'}

r_token = requests.get(val_url, auth=HTTPBasicAuth(val_user, val_pass), headers=val_headers)

val_token = r_token.headers['x-csrf-token']
val_cookies = r_token.cookies

r_token.headers

{'set-cookie': 'sap-usercontext=sap-client=100; path=/, sap-XSRF_M09_100=HThnrZCpFoEdQ5hR34OGnA%3d%3d20200717142517xNKwTgndh4IcoQU9AqXCxLEHoGUR8q1YNTE_dbdx5L4%3d; path=/; secure; HttpOnly, TS01521bba=0116b3bc5fcd06d4bc8662c3f8a320d730c24eb15b3402073ef81d6e79e2363974b0a8bded159ca9e9e5cb4d7732a7b768fd20ab1a62e81d1d2b69fa6780b9adf88e299870fde405182a2ca26a237190b4d37b6d4f; Path=/', 'content-type': 'application/xml', 'content-length': '3761', 'x-csrf-token': 'HThnrZCpFoEdQ5hR34OGnA==', 'last-modified': 'Thu, 16 Jul 2020 14:32:52 GMT', 'cache-control': 'max-age=0', 'dataserviceversion': '2.0', 'sap-processing-info': 'ODataBEP=,crp=,RAL=,st=,MedCacheHub=Table,codeployed=X,softstate=', 'sap-server': 'true', 'sap-perf-fesrec': '121438.000000', 'content-encoding': 'gzip', 'x-content-type-options': 'nosniff', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'x-xss-protection': '1; mode=block'}

In [8]:
#IMPORTAR PRODUCTOS MÁS VENDIDOS (REDSHIFT - COMMERCE)
import pandas as pd
import difflib
import numpy as np
import pyodbc

import psycopg2

#CONEXION A RS
var_dbname = 'analitico'
var_host = '10.12.2.26'
var_port ='5439'
var_user ='rcocchella'
var_password = 'Belcorp2019'


connRS=psycopg2.connect(dbname= var_dbname, 
                        host= var_host, 
                        port= var_port, 
                        user= var_user,
                        password=var_password )

#BaseCampaign
BaseCampaign ='201918'

query_products ="""
--TABLA DE PRODUCTOS A RECOMENDAR
DROP TABLE IF EXISTS #TEMP_PRODUCTS;
CREATE TABLE #TEMP_PRODUCTOS AS 
SELECT TOP 100 * FROM 
(SELECT codsap, sum(can_unidades) can_unidades FROM fnc_customer.ecm_det_detalle_orden
WHERE aniomes >= '202001'
GROUP BY 1
ORDER BY 2 DESC);

SELECT * FROM #TEMP_PRODUCTOS;
"""

df_products = pd.DataFrame(pd.read_sql(query_products, connRS))

connRS.commit()
connRS.close()

df_products['ProductOrigin'] = "BELCORP_PRODUCT"
df_products['Aux_join'] = ""

df_products.head(3)

,codsap,can_unidades,Aux_join
0,200097893,2315,
1,200095060,1406,
2,200095340,901,


In [386]:
#IMPORTAR PRODUCTOS DE MAESTRO COMMERCE EN SMC
fp_contacts = r'd:\Users\rcocchella\Desktop\Temporales\Personalizacion SMC\External Recommendation\Contacts Commerce QA.csv'
dataset_delimiter =';' 
df_contacts = pd.DataFrame(pd.read_csv(fp_contacts, engine = "python", error_bad_lines = False, delimiter =dataset_delimiter)) 

df_contacts = df_contacts.rename(columns={'ï»¿ID Origin':'id_origin','ID':'id'})

print(df_contacts.shape)
df_contacts.head(3)

(334047, 2)


,id_origin,id
0,GIGYA_ID,1522e2957393423bab5ba0e50884504d
1,GIGYA_ID,dc164a19e35447df82b9a097b463196d
2,GIGYA_ID,a6ed2e11a78641c7bba30d486938e405


In [470]:
#IMPORTAR PRODUCTOS DE MAESTRO COMMERCE EN SMC
import pandas as pd

fp  = r'd:\Users\rcocchella\Desktop\Temporales\Personalizacion SMC\External Recommendation\Products Commerce SMC\MasterProduct.xlsx'

df_products =  pd.read_excel(fp)
df_products = df_products.rename(columns={'Product ID':'codsap'})

df_products = df_products[["codsap"]]

df_products = df_products.loc[df_products['codsap'].isin(list(['200088265','200093367']))] 

df_products = df_products.head(100)

df_products['ProductOrigin'] = "SAP_HYBRIS_PRODUCT"

df_products['Aux_join'] = ""

print(df_products.shape)
df_products.head(3)

(2, 3)


,codsap,ProductOrigin,Aux_join
290,200093367,SAP_HYBRIS_PRODUCT,
3501,200088265,SAP_HYBRIS_PRODUCT,


In [471]:
#CREAR LISTA DE CONTACTOS CUSTOM

import pandas as pd

data = {'id_origin':  ['EMAIL','EMAIL','EMAIL','EMAIL','EMAIL','SAP_HYBRIS_CONSUMER','EMAIL', 'EMAIL'],
        'id': ['rcocchella106@gmail.com','rcocchella105@gmail.com','rcocchella104@gmail.com','rcocchella103@gmail.com','rcocchella102@gmail.com','f81259b7-81dd-4340-ba60-d72b063f17ce','rcocchella100@gmail.com', 'rcocchella101@gmail.com']
        }

df_contacts = pd.DataFrame (data, columns = ['id_origin','id'])

df_contacts= df_contacts.iloc[[1]]

print(df_contacts.shape)
df_contacts.head(3)

(1, 2)


,id_origin,id
1,EMAIL,rcocchella105@gmail.com


In [472]:
#Asignar Numeracion a Contactos
df_contacts['ContactRank'] = df_contacts.sort_values(['id'], ascending=[True]) \
             .groupby(['id_origin']) \
             .cumcount() + 1

df_contacts['Aux_join'] = ""

df_contacts['ContactRank'] = 7

df_contacts.head(3)

,id_origin,id,ContactRank,Aux_join
1,EMAIL,rcocchella105@gmail.com,7,


In [473]:
df_contacts_products = pd.merge(df_contacts, df_products,
         left_on = 'Aux_join',right_on = 'Aux_join',
         how = 'outer',
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

df_contacts_products = df_contacts_products

print(df_contacts_products.shape)
df_contacts_products.head(3)

(2, 6)


,id_origin,id,ContactRank,Aux_join,codsap,ProductOrigin
0,EMAIL,rcocchella105@gmail.com,7,,200093367,SAP_HYBRIS_PRODUCT
1,EMAIL,rcocchella105@gmail.com,7,,200088265,SAP_HYBRIS_PRODUCT


In [474]:
df_contacts_products['randNumCol'] = np.random.randint(0,len(df_contacts_products),size=len(df_contacts_products))
    
df_contacts_products['RandomRank'] = df_contacts_products.sort_values(['randNumCol'], ascending=[True]) \
             .groupby(['id_origin','id']) \
             .cumcount() + 1

print(df_contacts_products.shape)

df_contacts_products.head(10)

(2, 8)


,id_origin,id,ContactRank,Aux_join,codsap,ProductOrigin,randNumCol,RandomRank
0,EMAIL,rcocchella105@gmail.com,7,,200093367,SAP_HYBRIS_PRODUCT,0,1
1,EMAIL,rcocchella105@gmail.com,7,,200088265,SAP_HYBRIS_PRODUCT,1,2


In [150]:
#CHRISTIAN! PRIMERO CREO UNA ENTIDAD DE RECOMENDACIÓN EXTERNA VACÍA (PARA LUEGO INSERTAR LAS RECOMENDACIONES ESPECÍFICAS 
#PARACADA CLIENTE)

#PAYLOAD FOR CREATE EXTERNAL RECOMENDATION WITHOUT CLUSTERS
#EXP_B: EXTERNAL PRODUCT BACTH

import sys
var_df_contacts_products_topn = df_contacts_products_topn
val_ExternalRecommendation = 'EXP_B6'
val_ExtRecmdnExternalVersion = 'V1'


val_ExtRecmdnExternalVersion = val_ExternalRecommendation + '_' + val_ExtRecmdnExternalVersion


df_contacts_unique = var_df_contacts_products_topn[["id_origin","id","ContactRank"]].drop_duplicates()
df_contacts_unique.reset_index(inplace=True)

CntctToRecmdnClstrAssgmt_txt =\
'{'+\
'"ExternalRecommendation": "'+val_ExternalRecommendation+'"'+\
',"ExternalRecommendationName": "External Product"'+\
',"ExternalRecommendationType": "01"'+\
',"RecommendationResultItemType": "13"'+\
', "to_ExternalRecmdnVersion": '+\
    '['+\
        '{'+\
    '"ExtRecmdnExternalVersion": "'+val_ExtRecmdnExternalVersion +'"'+\
    ',"ValidityStartDateTime": "2017-02-15T09:00:00.0000000"'+\
    ',"to_RecommendationCluster": ['

CntctToRecmdnClstrAssgmt_txt = CntctToRecmdnClstrAssgmt_txt + "]" +\
        '}'+\
    ']'+\
'}'
    
CntctToRecmdnClstrAssgmt_txt


val_payload = CntctToRecmdnClstrAssgmt_txt


filePath = r'd:\Users\rcocchella\Desktop\Temporales\Personalizacion SMC\External Recommendation\Aux XML Responses\json_test.json'

with open(filePath, mode = 'w', encoding= "utf-8") as f:
    f.write(CntctToRecmdnClstrAssgmt_txt)

val_payload

'{"ExternalRecommendation": "EXP_B6","ExternalRecommendationName": "External Product","ExternalRecommendationType": "01","RecommendationResultItemType": "13", "to_ExternalRecmdnVersion": [{"ExtRecmdnExternalVersion": "EXP_B6_V1","ValidityStartDateTime": "2017-02-15T09:00:00.0000000","to_RecommendationCluster": []}]}'

In [151]:
#API POST CREATE EXTERNAL RECOMENDATION WITHOUT CLUSTERS
import json

val_url= 'https://my300972-api.s4hana.ondemand.com/sap/opu/odata/sap/API_MKT_EXTERNAL_RECMDN_SRV/ExternalRecommendations'


val_headers = {'x-csrf-token': val_token,
              'Content-Type':'application/json',
               'Sap-cuan-asynchronousProcessing':'X',
               'Sap-cuan-SourceSystemId':'Test',
               'Sap-cuan-SourceSystemType':'EXT'
              }


val_payload = val_payload

response = requests.post(val_url,auth=HTTPBasicAuth(val_user, val_pass), headers=val_headers,cookies=val_cookies, data = val_payload)
response

<Response [201]>

In [417]:
#CHRISTIAN! LUEGO GENERO EL PAYLOAD DE LAS RECOMENDACIONES ESPECÍFICAS PARA CADA CLIENTE, AGRUPADAS EN UN CLUSTER)
#ESTA FUNCION CONTIENE LA LOGICA PARA GENERAR EL PAYLOAD, QUE LUEGO LLAMO EN EL SIGUIENTE CELL

def gen_payload_cluster_add(var_df_contacts_products_topn,val_ExternalRecommendation,val_ExtRecmdnExternalVersion):
    
    val_ExternalRecommendation = val_ExternalRecommendation
    val_ExtRecmdnExternalVersion = val_ExtRecmdnExternalVersion
    val_ExtRecmdnExternalVersion = val_ExternalRecommendation + '_' + val_ExtRecmdnExternalVersion
    
    df_contacts_products_topn = var_df_contacts_products_topn
    
    df_contacts_unique = df_contacts_products_topn[["id_origin","id","ContactRank"]].drop_duplicates()
    df_contacts_unique.reset_index(inplace=True)
    
    print(df_contacts_products_topn.shape)
    df_contacts_products_topn.head(10)
    # 1 a 2
    # 3 a 4
    # 5 a 6
    # 7 a 106
    #2:500
    #3:100
    #5:
    
    guid = '923797a0-30ed-a121-1700-87003d5b1945'
    
    CntctToRecmdnClstrAssgmt_txt = """--batchtest
Content-Type: multipart/mixed; boundary=changeset_9970-5898-d67d """ 
    
    
    for index_row, row in df_contacts_unique.iterrows():
        sys.stdout.write('\r'+str('{:1}'.format(str(index_row))))
            
        CntctToRecmdnClstrAssgmt_txt_aux = """

--changeset_9970-5898-d67d
Content-Type: application/http
Content-Transfer-Encoding: binary

POST ExternalRecmdnVersions(guid'"""+guid+"""')/to_RecommendationCluster?saml2=disabled HTTP/1.1
sap-contextid-accept: header
Accept: application/json
Accept-Language: en-US
DataServiceVersion: 2.0
MaxDataServiceVersion: 2.0
Content-Type: application/json
sap-cuan-asynchronousprocessing: X
sap-cuan-sourcesystemid: Test
sap-cuan-sourcesystemtype: EXT

"""
        
        #Generar arreglo de cluster con asignacioin de contactos
        CntctToRecmdnClstrAssgmt_txt_aux=CntctToRecmdnClstrAssgmt_txt_aux+\
        '{'+\
            '"RecommendationCluster": "'+val_ExtRecmdnExternalVersion+"_"+str(row["ContactRank"]) +'"'+\
            ',"IsFallbackRecmdnCluster": '+"false" +\
            ',"to_CntctToRecmdnClstrAssgmt": '+\
                '['+\
                    '{'+\
                    '"InteractionContactId": "'+str(row["id"]) +'"'+\
                    ',"InteractionContactOrigin": "'+str(row["id_origin"]) +'"'+\
                    '}'+\
                ']'
    
        df_contacts_unique_product = df_contacts_products_topn.loc[(df_contacts_products_topn.id == row.id)]
        df_contacts_unique_product.reset_index(inplace=True)
        
        RecmdnClstrItemAssgmt_txt = ',"to_RecmdnClstrItemAssgmt": ['
        RecmdnClstrItemAssgmt_txt_aux = ''
            
        for index_row_p, row_p in df_contacts_unique_product.iterrows():
            
            RecmdnClstrItemAssgmt_txt_aux = \
            '{'+\
            '"RecommendationResultItemType": "13"'+\
            ',"RecommendationResultItem": "'+str(row_p["codsap"]) +'"'+\
            ',"RecommendationResultItemOrigin": "'+str(row_p["ProductOrigin"]) +'"'+\
            ',"RecommendationItemScore": '+str(row_p["RandomRank"]) +\
            '}'
            
            if index_row_p == 0:
                sep_p = ''
            else:
                sep_p =','
    
            RecmdnClstrItemAssgmt_txt = RecmdnClstrItemAssgmt_txt + sep_p + RecmdnClstrItemAssgmt_txt_aux
        
        RecmdnClstrItemAssgmt_txt = RecmdnClstrItemAssgmt_txt + "]}" 
        
        #Añadir al cluster 
        CntctToRecmdnClstrAssgmt_txt_aux = CntctToRecmdnClstrAssgmt_txt_aux + RecmdnClstrItemAssgmt_txt
        
        CntctToRecmdnClstrAssgmt_txt = CntctToRecmdnClstrAssgmt_txt + CntctToRecmdnClstrAssgmt_txt_aux
    
    
    CntctToRecmdnClstrAssgmt_txt = CntctToRecmdnClstrAssgmt_txt + """

--changeset_9970-5898-d67d--
 
--batchtest--"""
    
    return CntctToRecmdnClstrAssgmt_txt

In [475]:
#CHRISTIAN! LUEGO GENERO Y POSTEO LOS PAYLOADS USANDO LA FUNCION PREVIA, CON UN LIMITE DE 4000 CLIENTES (PUEDE SER HASTA 5000
#SEGUN RECOMENDACION DE PYXIES)

#BATCHES
import os
import math
import datetime 


n_products = 10
n_contacts_li = 1
n_contacts_ls = 2
batch_size = 4000

val_ExternalRecommendation = 'EXP_B6'
val_ExtRecmdnExternalVersion = 'V3'

for i in range(1,math.ceil((n_contacts_ls-n_contacts_li)/batch_size)+1):
    print(i)
    
    df_contacts_products_topn = df_contacts_products.loc[(df_contacts_products.RandomRank <= n_products)]
    
    df_contacts_products_topn = df_contacts_products_topn.loc[(df_contacts_products_topn.ContactRank >= n_contacts_li + (i-1)*batch_size)]
    df_contacts_products_topn = df_contacts_products_topn.loc[(df_contacts_products_topn.ContactRank < n_contacts_li + (i)*batch_size)]
    
    CntctToRecmdnClstrAssgmt_txt = gen_payload_cluster_add(df_contacts_products_topn,val_ExternalRecommendation,val_ExtRecmdnExternalVersion)
    
    val_url= "https://my300972-api.s4hana.ondemand.com/sap/opu/odata/sap/API_MKT_EXTERNAL_RECMDN_SRV/$batch"

    val_headers = {'x-csrf-token': val_token,
                  'Content-Type':'multipart/mixed; boundary=batchtest',
                   'Sap-cuan-asynchronousProcessing':'X',
                   'Sap-cuan-SourceSystemId':'Test',
                   'Sap-cuan-SourceSystemType':'EXT'
                  }
    
    
    val_payload = CntctToRecmdnClstrAssgmt_txt
    
    
    response = requests.post(val_url,auth=HTTPBasicAuth(val_user, val_pass), headers=val_headers,cookies=val_cookies, data = val_payload)
    
    print(response)
    
    datetime_now = str.replace(str.replace(str(datetime.datetime.now()),':','-'),'.','-')
    
    filename = r'd:\Users\rcocchella\Desktop\Temporales\Personalizacion SMC\External Recommendation\Aux XML Responses\response_text'+datetime_now+'.xml'
    with open(filename, mode='w') as f:
        f.write(response.text)


1
(2, 8)
0<Response [202]>


In [476]:
response.text

'--DFA7847498CE84795DA8FF19515127C10\r\nContent-Type: multipart/mixed; boundary=DFA7847498CE84795DA8FF19515127C11\r\nContent-Length:      1892\r\n\r\n--DFA7847498CE84795DA8FF19515127C11\r\nContent-Type: application/http\r\nContent-Length: 1723\r\ncontent-transfer-encoding: binary\r\n\r\nHTTP/1.1 201 Created\r\nContent-Type: application/json\r\nContent-Length: 1455\r\nlocation: https://my300972-api.s4hana.ondemand.com/sap/opu/odata/sap/API_MKT_EXTERNAL_RECMDN_SRV/RecommendationClusters(guid\'00000000-0000-0000-0000-000000000000\')\r\ndataserviceversion: 2.0\r\n\r\n{"d":{"__metadata":{"id":"https://my300972-api.s4hana.ondemand.com/sap/opu/odata/sap/API_MKT_EXTERNAL_RECMDN_SRV/RecommendationClusters(guid\'00000000-0000-0000-0000-000000000000\')","uri":"https://my300972-api.s4hana.ondemand.com/sap/opu/odata/sap/API_MKT_EXTERNAL_RECMDN_SRV/RecommendationClusters(guid\'00000000-0000-0000-0000-000000000000\')","type":"API_MKT_EXTERNAL_RECMDN_SRV.RecommendationClustersType"},"RecommendationClu